<a href="https://www.kaggle.com/code/jeff125/top-100-anime-movies?scriptVersionId=111653764" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/top-100-anime-movies-imdb/top_100_anime_movie.csv


In [2]:
data = pd.read_csv('/kaggle/input/top-100-anime-movies-imdb/top_100_anime_movie.csv')

In [3]:
data.head(10)

,rank,title,rating,votes,year,minutes,genre,gross
0,1,Ramayana: The Legend of Prince Rama,9.2,"7,549",1993,97.0,"Animation, Action, Adventure",NaN
1,2,Spirited Away,8.6,"756,112",2001,125.0,"Animation, Adventure, Family",$10.06M
2,3,Meiji Tokyo Renka Movie: Yumihari no Serenade,8.5,39,2015,60.0,"Animation, Fantasy, Romance",NaN
3,4,"Natsu e no tunnel, Sayonara no deguchi",8.5,23,2022,83.0,Animation,NaN
4,5,Attack on Titan: Chronicle,8.5,"10,421",2020,122.0,"Animation, Action, Adventure",NaN
5,6,Gintama: The Final,8.5,"2,381",2021,104.0,"Animation, Action, Comedy",NaN
6,7,Grave of the Fireflies,8.5,"275,743",1988,89.0,"Animation, Drama, War",NaN
7,8,Attack on Titan: The Roar of Awakening,8.4,"5,376",2018,120.0,"Animation, Action, Fantasy",NaN
8,9,Your Name.,8.4,"265,332",2016,106.0,"Animation, Drama, Fantasy",$5.02M
9,10,Princess Mononoke,8.4,"392,732",1997,134.0,"Animation, Action, Adventure",$2.38M


In [4]:
print(data.shape)
print("-----------" * 5)
print(data.dtypes)
print("-----------" * 5)
print(data.describe()) 
print("-----------" * 5)
print(data.info()) 
print("-----------" * 5)
print(data.isnull())

(100, 8)
-------------------------------------------------------
rank         int64
title       object
rating     float64
votes       object
year         int64
minutes    float64
genre       object
gross       object
dtype: object
-------------------------------------------------------
             rank      rating         year     minutes
count  100.000000  100.000000   100.000000   96.000000
mean    50.500000    8.001000  2007.810000  100.114583
std     29.011492    0.254056    12.233168   25.358013
min      1.000000    7.700000  1979.000000   45.000000
25%     25.750000    7.800000  1997.750000   84.750000
50%     50.500000    8.000000  2013.000000  103.500000
75%     75.250000    8.100000  2018.000000  117.500000
max    100.000000    9.200000  2022.000000  162.000000
-------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   -------

In [5]:
print(data['gross'].isnull().sum())

80


# **TOP Gross Data**

In [6]:
df = data.dropna()
gross_data = df['gross'].replace('[$ M]',r'',regex=True)

sort_gross_data = gross_data.astype({'gross':'float'})
results=[]

for i in sort_gross_data.sort_values(ascending=False):
    results.append(round(i*1000000))
    
df.drop(columns=['gross'])
df['gross'] = results


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [7]:
import plotly.express as px

fig = px.bar(df , x=df['title'], y=df['gross'],text_auto=True,color=df['rating'] ,height=800)
fig.update_yaxes(tickformat='$,')
fig.update_xaxes(tickangle=90)
fig.update_traces(textfont_size=18, textangle=0, textposition="outside", cliponaxis=False)
fig.show()

In [8]:
genre_data = df['genre'].replace('Animation,',r'',regex=True)
vote_data = df['votes'].replace(',',r'',regex=True)

df.drop(columns=['genre','votes'])

votes_data = vote_data.astype({'votes':'int'})

df['genre'] = genre_data
df['votes'] = votes_data

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# **Correlation Between Genre, Rating and Gross**

In [9]:
fig = px.scatter(df ,x="rating", y="gross", size="votes", color="genre",
           hover_name="title",log_x=True, size_max=50)

fig.update_yaxes(tickformat='$,')

fig.show()

# **Correlation Rating for Duration by Year**

In [10]:
fig = px.scatter(data ,x="year", y="minutes", color = "rating",
           hover_name="title",log_x=True, size_max=50)

fig.show()

# **Distribution plot of Duration and Rating**

In [11]:
fig = px.scatter(data ,x="rating", y="minutes", 
           hover_name="title",log_x=True, size_max=50)

fig.show()